In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current 
df = pd.read_csv("/kaggle/input/worldnews-on-reddit/reddit_worldnews_start_to_2016-11-22.csv", usecols=['title'], dtype={'title': 'str'})
print("Data loaded")
print(df)


Data loaded
                                                    title
0                       Scores killed in Pakistan clashes
1                        Japan resumes refuelling mission
2                         US presses Egypt on Gaza border
3            Jump-start economy: Give health care to all 
4         Council of Europe bashes EU&UN terror blacklist
...                                                   ...
509231   Heil Trump : Donald Trump s  alt-right  white...
509232  There are people speculating that this could b...
509233          Professor receives Arab Researchers Award
509234  Nigel Farage attacks response to Trump ambassa...
509235  Palestinian wielding knife shot dead in West B...

[509236 rows x 1 columns]
